In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Function to scrape LinkedIn data (Note: You must ensure you're not violating LinkedIn's terms of service)
def scrape_linkedin_jobs(query, location, max_results=10):
    url = f"https://www.linkedin.com/jobs/search/?keywords={query}&location={location}&trk=public_jobs_jobs-search-bar_search-submit&start=0"

    # Simulating a real browser visit by setting user-agent
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    job_listings = []

    # Find all job posting elements
    jobs = soup.find_all('li', {'class': 'result-card'})

    for job in jobs[:max_results]:
        title = job.find('h3', {'class': 'result-card__title'}).text.strip()
        company = job.find('h4', {'class': 'result-card__subtitle'}).text.strip()
        location = job.find('span', {'class': 'job-result-card__location'}).text.strip()

        # Add more fields as necessary
        job_listings.append({
            'Job Title': title,
            'Company': company,
            'Location': location,
        })

    return pd.DataFrame(job_listings)

# Load dataset
df = pd.read_csv("job_descriptions.csv")  

# Combine the LinkedIn data with your dataset for analysis (merge on Job Title, or any relevant feature)
df_combined = pd.concat([df, linkedin_data], ignore_index=True)

# Keep relevant features for prediction
features = ['Experience']

target = 'Processed Salary'  # Target variable is now the processed salary

# Function to process Salary Range
def process_salary_range(salary):
    salary = salary.replace(",", "").lower()
    match = re.match(r"(\d+)k-(\d+)k", salary)
    if match:
        low, high = map(int, match.groups())
        return (low + high) / 2
    match = re.match(r"(\$?\d+)k-(\$?\d+)k", salary)
    if match:
        low, high = match.groups()
        low = int(low.replace("$", ""))
        high = int(high.replace("$", ""))
        return (low + high) / 2
    match = re.match(r"(\d+)k\s?-\s?(\d+)k", salary)
    if match:
        low, high = map(int, match.groups())
        return (low + high) / 2
    print(f"Invalid salary format: {salary}")
    return None

# Apply the salary range processing
df_combined['Processed Salary'] = df_combined['Salary Range'].apply(process_salary_range)

# Drop rows with missing processed salary values
df_combined = df_combined.dropna(subset=['Processed Salary'])

# Function to process Experience Range (e.g., "5 to 9 Years")
def process_experience(experience, work_type):
    # Remove "years" and extra spaces
    experience = str(experience).lower().replace(" years", "").strip()

    # Process intern work types: use the first value in the range
    if 'intern' in work_type.lower():
        match = re.match(r"(\d+)\s?to\s?(\d+)", experience)
        if match:
            low, high = map(int, match.groups())
            return low  # Interns use the lower bound
    # Process full-time work types: use the second value in the range
    if 'full time' in work_type.lower():
        match = re.match(r"(\d+)\s?to\s?(\d+)", experience)
        if match:
            low, high = map(int, match.groups())
            return high  # Full-time uses the higher bound
    # For other work types, use the average
    match = re.match(r"(\d+)\s?to\s?(\d+)", experience)
    if match:
        low, high = map(int, match.groups())
        return (low + high) / 2  # Average for other work types
    # In case of single values (e.g., "5 years"), just return that value
    match = re.match(r"(\d+)", experience)
    if match:
        return int(match.group(1))  # Return single value if no range

    return np.nan  # Return NaN if the format is unexpected

# Apply the experience range processing
df_combined['Processed Experience'] = df_combined.apply(lambda row: process_experience(row['Experience'], row['Work Type']), axis=1)

# Drop rows with missing processed experience values
df_combined = df_combined.dropna(subset=['Processed Experience'])

# Feature Engineering: Create new features
df_combined['Qualifications_Length'] = df_combined['Qualifications'].apply(lambda x: len(str(x).split()))  # Number of words in qualifications

# Ensure 'Experience' is processed and cleaned
df_combined['Experience'] = df_combined['Experience'].apply(lambda x: process_experience(x, 'Unknown'))  # Just a placeholder to ensure it's processed
df_combined['Experience'] = pd.to_numeric(df_combined['Experience'], errors='coerce')

# Encode categorical features
label_encoders = {}
for col in df_combined.select_dtypes(include=['object']).columns:
    if col not in ['Salary Range', 'Experience']:  # Skip 'Salary Range' and 'Experience' as they are already processed
        le = LabelEncoder()
        df_combined[col] = le.fit_transform(df_combined[col].astype(str))
        label_encoders[col] = le

# Ensure that all features are numeric before scaling
numeric_features = ['Processed Experience', 'latitude', 'longitude']

# Convert any non-numeric values in 'Processed Experience' to NaN
df_combined['Processed Experience'] = pd.to_numeric(df_combined['Processed Experience'], errors='coerce')

# Standardize numerical features
scaler = StandardScaler()
df_combined[numeric_features] = scaler.fit_transform(df_combined[numeric_features])

# Split data into features (X) and target (y)
X = df_combined[features + ['Qualifications_Length', 'Processed Experience']]
y = df_combined[target]  # Using 'Processed Salary' as the target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on both training and test data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Define a custom accuracy function for regression
def regression_accuracy(y_true, y_pred, tolerance=0.1):
    """
    Calculates the percentage of predictions within a certain tolerance of the true value.
    :param y_true: Array of true target values.
    :param y_pred: Array of predicted target values.
    :param tolerance: Tolerance as a fraction of the true value (default 10%).
    :return: Accuracy as a percentage.
    """
    within_tolerance = np.abs(y_true - y_pred) <= (tolerance * y_true)
    accuracy = np.mean(within_tolerance) * 100  # Convert to percentage
    return accuracy

# Evaluate on training data
evaluate_performance(y_train, y_train_pred, "Training")
training_accuracy = regression_accuracy(y_train, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.2f}%")

# Evaluate on test data
evaluate_performance(y_test, y_test_pred, "Test")
test_accuracy = regression_accuracy(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Calculate RMSE for training and test sets
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Training RMSE: {rmse_train:.2f}")
print(f"Test RMSE: {rmse_test:.2f}")



Training Data Performance:
Mean Squared Error: 56.60
Training Accuracy: 65.53%
Test Data Performance:
Mean Squared Error: 56.88
Test Accuracy: 66.31%
Training RMSE: 7.52
Test RMSE: 7.54
